In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Reliance_stock_data.csv')
print(df)

         Date      Open     High     Low     Close
0     04-Jun-10  1031.0  1036.00  1022.45  1031.20
1     07-Jun-10  1015.0  1015.00  1000.00  1007.75
2     08-Jun-10  1009.7  1012.70   995.10   996.40
3     09-Jun-10  1000.2  1015.00  1000.00  1007.05
4     10-Jun-10  1010.0  1017.45  1007.15  1015.35
...         ...     ...      ...      ...      ...
3246  12-Jun-23  2475.5  2489.20  2465.00  2484.15
3247  13-Jun-23  2495.0  2523.45  2486.25  2520.85
3248  14-Jun-23  2521.0  2555.00  2517.00  2551.85
3249  15-Jun-23  2551.8  2562.55  2535.50  2552.55
3250  16-Jun-23  2560.2  2582.40  2560.20  2577.40

[3251 rows x 5 columns]


In [3]:
print(df.columns)

Index(['Date  ', 'Open', 'High', 'Low ', 'Close'], dtype='object')


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [5]:
selected_features = ['Open', 'Close', 'High', 'Low ']
data = df[selected_features].values
data = data.astype(float)

scaler = MinMaxScaler()
data = scaler.fit_transform(data)

X = []
y = []
for i in range(2, len(data)):
    X.append(np.concatenate((data[i-2], data[i-1])))
    y.append(data[i])

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
model = Sequential()
model.add(Dense(2, activation='sigmoid', input_dim=8))
model.add(Dense(2, activation='sigmoid'))
model.add(Dense(4, activation='linear'))

In [13]:
model.compile(optimizer=Adam(learning_rate=0.1), loss='mse')
early_stopping = EarlyStopping(monitor='val_loss', patience=10000, restore_best_weights=True)


In [15]:
model.fit(X_train, y_train, epochs=10000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/10000
82/82 [==============================] - 0s 4ms/step - loss: 2.0263e-04 - val_loss: 2108056.5000
Epoch 2/10000
82/82 [==============================] - 0s 4ms/step - loss: 2.3667e-04 - val_loss: 2108050.5000
Epoch 3/10000
82/82 [==============================] - 0s 4ms/step - loss: 2.4092e-04 - val_loss: 2108090.7500
Epoch 4/10000
82/82 [==============================] - 0s 4ms/step - loss: 2.5076e-04 - val_loss: 2108005.2500
Epoch 5/10000
82/82 [==============================] - 0s 4ms/step - loss: 2.3629e-04 - val_loss: 2107994.5000
Epoch 6/10000
82/82 [==============================] - 0s 4ms/step - loss: 3.8430e-04 - val_loss: 2107918.5000
Epoch 7/10000
82/82 [==============================] - 0s 4ms/step - loss: 7.1000e-04 - val_loss: 2108010.2500
Epoch 8/10000
82/82 [==============================] - 0s 3ms/step - loss: 2.4639e-04 - val_loss: 2108063.0000
Epoch 9/10000
82/82 [==============================] - 0s 4ms/step - loss: 2.3864e-04 - val_loss: 2108042.2500
E

KeyboardInterrupt: ignored

In [16]:
loss = model.evaluate(X_test, y_test)
print("Mean Squared Error:", loss)

21/21 [==============================] - 0s 2ms/step - loss: 2108034.2500
Mean Squared Error: 2108034.25


In [10]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error,accuracy_score

y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", mape)



21/21 [==============================] - 0s 2ms/step
R^2 Score: 0.9988518130345663
MAPE: 0.011075083957524429
